In [1]:
from datetime import datetime, timedelta, date
from itertools import cycle
# from lunarcalendar import Converter, Solar, Lunar, DateNotExist
import re
import pandas as pd

In [114]:
preproc = pd.read_excel("data/preproc.xlsx", header=None)

preproc

,0,1
0,1 (ян),甲子
1,NaN,Цзя Цзы
2,Деревянная Крыса,NaN
3,2 (инь),乙丑
4,NaN,И Чоу
...,...,...
175,NaN,Жэнь Сюй
176,Водяная Собака,NaN
177,60 (инь),癸亥
178,NaN,Гуй Хай


In [115]:
preproc[0] = preproc[0].fillna(" ")
preproc[1] = preproc[1].fillna("+")
preproc = preproc.T
preproc

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,1 (ян),,Деревянная Крыса,2 (инь),,Деревянный Бык,3 (ян),,Огненный Тигр,4 (инь),...,Металлическая Обезьяна,58 (инь),,Металлический Петух,59 (ян),,Водяная Собака,60 (инь),,Водяная Свинья
1,甲子,Цзя Цзы,+,乙丑,И Чоу,+,丙寅,Бин Инь,+,丁卯,...,+,辛酉,Синь Ю,+,壬戌,Жэнь Сюй,+,癸亥,Гуй Хай,+


In [122]:
s = ","
for i in preproc:
    s += preproc[i].str.cat(sep=",") + ","
s

',1 (ян),甲子 , ,Цзя Цзы,Деревянная Крыса,+,2 (инь),乙丑   , ,И Чоу,Деревянный Бык,+,3 (ян),丙寅, ,Бин Инь,Огненный Тигр,+,4 (инь),丁卯, ,Дин Мао,Огненный Кролик,+,5 (ян),戊辰    , ,У Чэнь,Земляой Дракон,+,6 (инь),己巳 , ,Цзи Сы,Земляная Змея,+,7 (ян),庚午   , ,Гэн У,Металлическая Лошадь,+,8 (инь),辛未  , ,Синь Вэй,Металлическая Коза,+,9 (ян),壬申 , ,Жэнь Шэнь,Водяная Обезьяна,+,10 (инь),癸酉    , ,Гуй Ю,Водяной Петух,+,11 (ян),甲戌   \xa0, ,Цзя Сюй,Деревянная Собака,+,12 (инь),乙亥   , ,И Хай,Деревянная Свинья,+,13 (ян),丙子, ,Бин Цзы,Огненная Крыса,+,14 (инь),丁丑 , ,Дин Чоу,Огненный Бык,+,15 (ян),戊寅, ,У Инь,Земляной Тигр,+,16 (инь),己卯, ,Цзи Мао,Земляной Кролик,+,17 (ян),庚辰, ,Гэн Чэнь,Металлический Дракон,+,18 (инь),辛巳, ,Синь Сы,Металлическая Змея,+,19 (ян),壬午, ,Жэнь У,Водяная Лошадь,+,20 (инь),癸未, ,Гуй Вэй,Водяная Коза,+,21 (ян),甲申, ,Цзя Шэнь,Деревянная Обезьяна,+,22 (инь),乙酉 , ,И Ю,Деревянный Петух,+,23 (ян),丙戌, ,Бин Сюй,Огненная Собака,+,24 (инь),丁亥 , ,Дин Хай,Огненная Свинья,+,25 (ян),戊子 , ,У Цзы,Земляная  

In [123]:
lst = s.split("+")
sdf = []
for l in lst:
    sdf.append(l.split(","))

In [124]:
sdf

[['', '1 (ян)', '甲子 ', ' ', 'Цзя Цзы', 'Деревянная Крыса', ''],
 ['', '2 (инь)', '乙丑   ', ' ', 'И Чоу', 'Деревянный Бык', ''],
 ['', '3 (ян)', '丙寅', ' ', 'Бин Инь', 'Огненный Тигр', ''],
 ['', '4 (инь)', '丁卯', ' ', 'Дин Мао', 'Огненный Кролик', ''],
 ['', '5 (ян)', '戊辰    ', ' ', 'У Чэнь', 'Земляой Дракон', ''],
 ['', '6 (инь)', '己巳 ', ' ', 'Цзи Сы', 'Земляная Змея', ''],
 ['', '7 (ян)', '庚午   ', ' ', 'Гэн У', 'Металлическая Лошадь', ''],
 ['', '8 (инь)', '辛未  ', ' ', 'Синь Вэй', 'Металлическая Коза', ''],
 ['', '9 (ян)', '壬申 ', ' ', 'Жэнь Шэнь', 'Водяная Обезьяна', ''],
 ['', '10 (инь)', '癸酉    ', ' ', 'Гуй Ю', 'Водяной Петух', ''],
 ['', '11 (ян)', '甲戌   \xa0', ' ', 'Цзя Сюй', 'Деревянная Собака', ''],
 ['', '12 (инь)', '乙亥   ', ' ', 'И Хай', 'Деревянная Свинья', ''],
 ['', '13 (ян)', '丙子', ' ', 'Бин Цзы', 'Огненная Крыса', ''],
 ['', '14 (инь)', '丁丑 ', ' ', 'Дин Чоу', 'Огненный Бык', ''],
 ['', '15 (ян)', '戊寅', ' ', 'У Инь', 'Земляной Тигр', ''],
 ['', '16 (инь)', '己卯', ' ', 'Цзи Ма

In [129]:
df = pd.DataFrame(sdf[:-1])
df.drop([0, 3, 6], axis=1, inplace=True)

In [131]:
df[:3]

,1,2,4,5
0,1 (ян),甲子,Цзя Цзы,Деревянная Крыса
1,2 (инь),乙丑,И Чоу,Деревянный Бык
2,3 (ян),丙寅,Бин Инь,Огненный Тигр


In [145]:
df[1] = df[1].str.replace("(", '')\
    .str.replace(")", "")\
    .str.replace("\d*", "", regex=True)

In [153]:
df.index = df.index + 1

In [154]:
df

,1,2,4,5
1,ян,甲子,Цзя Цзы,Деревянная Крыса
2,инь,乙丑,И Чоу,Деревянный Бык
3,ян,丙寅,Бин Инь,Огненный Тигр
4,инь,丁卯,Дин Мао,Огненный Кролик
5,ян,戊辰,У Чэнь,Земляой Дракон
6,инь,己巳,Цзи Сы,Земляная Змея
7,ян,庚午,Гэн У,Металлическая Лошадь
8,инь,辛未,Синь Вэй,Металлическая Коза
9,ян,壬申,Жэнь Шэнь,Водяная Обезьяна
10,инь,癸酉,Гуй Ю,Водяной Петух


In [161]:
df.to_csv("data/cicle.csv", index=False)


In [162]:
df = pd.read_csv("data/cicle.csv")
df[:2]

,Лунный_день,инь_ян,Иероглиф,Название_Китай,Название_Русский
0,1,ян,甲子,Цзя Цзы,Деревянная Крыса
1,2,инь,乙丑,И Чоу,Деревянный Бык
